# Data Preparation

## Imports

In [107]:
import pandas as pd
import numpy as np
import ast
import pytz
import re
import matplotlib.pyplot as plt
import xgboost as xgb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")




## Read files and concatenate ID orderbooks

In [57]:
# 01.01.2024 00:00 - 31.12.2024 23:00 (CET)
da_prices_df = pd.read_parquet('Data\da_prices_2024.parquet')

In [ ]:
# 01.01.2024 01:00 - 01.04.2024 21:00 (UCT)
orderbook_q1 = pd.read_parquet('Data\orderbook_q1.parquet')

# 01.04.2024 00:00 - 01.07.2024 21:00 (UCT)
orderbook_q2 = pd.read_parquet('Data\orderbook_q2.parquet')

# 01.07.2024 00:00 - 01.10.2024 21:00 (UCT)
orderbook_q3 = pd.read_parquet('Data\orderbook_q3.parquet')

# 01.10.2024 00:00 - 01.01.2025 22:00 (UCT)
orderbook_q4 = pd.read_parquet('Data\orderbook_q4.parquet')

KeyboardInterrupt: 

In [27]:
orderbook_df = pd.concat([orderbook_q1, orderbook_q2, orderbook_q3, orderbook_q4])
orderbook_df.to_parquet('Data/orderbook.parquet')

In [34]:
orderbook_df.dtypes

contractId                     object
contractName                   object
dlvryStart        datetime64[ns, UTC]
dlvryEnd          datetime64[ns, UTC]
dlvryAreaId                     int64
marketId                       object
area                           object
recorded                       object
time              datetime64[ns, UTC]
orderId_bid                    object
orderId_ask                    object
orderPrice_bid                 object
orderPrice_ask                 object
orderQty_bid                   object
orderQty_ask                   object
dtype: object

In [33]:
da_prices_df.dtypes

Delivery Start (CET)    datetime64[ns, UTC]
Delivery End (CET)      datetime64[ns, UTC]
NO3 Price (EUR)                     float64
dtype: object

## Convert columns to datetime

In [30]:
# The coulumns dlvryStart, dlvryEnd and time are in string format. We need to convert them to datetime format.|
orderbook_df['dlvryStart'] = pd.to_datetime(orderbook_df['dlvryStart'])
orderbook_df['dlvryEnd'] = pd.to_datetime(orderbook_df['dlvryEnd'])
orderbook_df['time'] = pd.to_datetime(orderbook_df['time'])

In [58]:
# Convert string columns to datetime objects (without timezone info initially)
da_prices_df['Delivery Start (CET)'] = pd.to_datetime(da_prices_df['Delivery Start (CET)'], format="%d.%m.%Y %H:%M:%S")
da_prices_df['Delivery End (CET)'] = pd.to_datetime(da_prices_df['Delivery End (CET)'], format="%d.%m.%Y %H:%M:%S")

# Localize to CET timezone
cet = pytz.timezone('CET')
da_prices_df['Delivery Start (CET)'] = da_prices_df['Delivery Start (CET)'].dt.tz_localize(cet, ambiguous='infer')
da_prices_df['Delivery End (CET)'] = da_prices_df['Delivery End (CET)'].dt.tz_localize(cet, ambiguous='infer')

# Convert from CET to UCT
da_prices_df['Delivery Start (UTC)'] = da_prices_df['Delivery Start (CET)'].dt.tz_convert('UTC')
da_prices_df['Delivery End (UTC)'] = da_prices_df['Delivery End (CET)'].dt.tz_convert('UTC')

In [59]:
da_prices_df.dtypes

Delivery Start (CET)    datetime64[ns, CET]
Delivery End (CET)      datetime64[ns, CET]
NO3 Price (EUR)                     float64
Delivery Start (UTC)    datetime64[ns, UTC]
Delivery End (UTC)      datetime64[ns, UTC]
dtype: object

In [60]:
# Remove the CET columns
da_prices_df = da_prices_df.drop(columns=['Delivery Start (CET)', 'Delivery End (CET)'])

In [51]:
orderbook_df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,orderId_ask,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask
0,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:00:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...","{'X5385371204','X5385374175','X5385353666','X5...","{2247,2100,2000,2000,1751,1443,1212,1205,1205,...","{2350,2378,2409,2497,2514,2559,2590,2600,2657,...","{500,5000,800,1600,1000,700,5000,5200,800,2200}","{10000,2000,5000,1000,200,500,300,10000,4000,500}"
1,NX_426238,PH-20240101-04,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:56.947 +0000,2023-12-31 23:00:00+00:00,"{'X5385372330','X5385372495','X5385371311','X5...","{'X5385372323','X5385372333','X5385373948','X5...","{2061,2061,2051,2000,2000,2000,1920,1467,1000,...","{2251,2251,2262,2299,2300,2433,2447,2448,2551,...","{1900,3000,100,5000,800,1500,400,500,1000,1000}","{200,500,200,3000,10000,500,3600,15000,4000,4000}"
2,NX_426245,PH-20240101-05,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:00.755 +0000,2023-12-31 23:00:00+00:00,"{'X5385359141','X5385367235','X5385367212','X5...","{'X5385364822','X5385364828','X5385364821','X5...","{2000,1950,1927,1917,1916,1320,1319,1319,1319,...","{2127,2127,2129,2150,2200,2400,2400,2401,2401,...","{100,100,1200,9000,2000,4500,1600,500,1000,1000}","{200,5000,400,10000,2000,4000,5000,500,15000,400}"
3,NX_426252,PH-20240101-06,2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:06.307 +0000,2023-12-31 23:00:00+00:00,"{'X5385371584','X5385371589','X5385371590','X5...","{'X5385363190','X5385363118','X5383614807','X5...","{1574,1574,1574,1574,1574,1574,1424,1423,1000,...","{1798,1800,1875,2050,2123,2423,2504,2550,2670,...","{1500,5000,1000,2000,1000,5000,2000,5000,1000,...","{10000,10000,10000,5000,15000,1000,1900,10000,..."
4,NX_426260,PH-20240101-07,2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:10.161 +0000,2023-12-31 23:00:00+00:00,"{'X5384463871','X5385167819','X5385169379','X5...","{'X5385370820','X5385371596','X5385371593','X5...","{1748,1748,1748,1748,1746,1520,1500,1500,1362,...","{2039,2089,2090,2262,2462,2562,2670,2700,2710,...","{4900,1000,1000,500,5000,1000,2000,1300,5000,5...","{500,1000,10000,15000,10000,1000,5000,5000,100..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717815,NX_496910,PH-20250101-20,2025-01-01 18:00:00+00:00,2025-01-01 19:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:44.288 +0000,2024-12-31 22:59:00+00:00,"{'X10233448499','X10234051602','X10234052934',...","{'X10232980063','X10228851570','X10232667440',...","{980,828,828,809,743,721,721,658,575,545}","{2775,2780,2780,2810,2810,2870,2900,3500}","{3900,4000,5000,5000,5000,5000,4000,800,2000,200}","{200,8000,5000,5000,5000,10000,2700,10000}"
2717816,NX_496918,PH-20250101-21,2025-01-01 19:00:00+00:00,2025-01-01 20:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:48.436 +0000,2024-12-31 22:59:00+00:00,"{'X10233744528','X10233744531','X10233744825',...","{'X10228851573','X10229998186','X10229998185',...","{513,513,509,500,498,498,497,394,377,376}","{2780,2780,2810,2810,2820,2870,2900,2900,2950,...","{5000,5000,5000,3900,5000,5000,300,2500,5000,6...","{6400,5000,5000,5000,10000,10000,1900,5000,700..."
2717817,NX_496925,PH-20250101-22,2025-01-01 20:00:00+00:00,2025-01-01 21:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:52.589 +0000,2024-12-31 22:59:00+00:00,"{'X10234238392','X10234238451','X10234238459',...","{'X10228851577','X10229959084','X10225892381',...","{645,645,645,645,645,616,616,574,509,505}","{2780,2780,2800,2810,2810,2820,2870,2900,2900,...","{100,5000,5000,5000,3700,5000,5000,5000,5000,3...","{5400,5000,1000,5000,5000,10000,10000,5000,700..."
2717818,NX_496932,PH-20250101-23,2025-01-01 21:00:00+00:00,2025-01-01 22:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:58.555 +0000,2024-12

## Merge ID orderbook with DA prices

In [61]:
# Merge the ID orderbook with the DA prices.
df = pd.merge(orderbook_df, da_prices_df, left_on='dlvryStart', right_on='Delivery Start (UTC)', how='left')

In [ ]:
df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,orderId_ask,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask,Delivery Start (CET),Delivery End (CET),NO3 Price (EUR)
0,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:00:00.000 +0000,"{'X5385339387','X5385339295','X5385350510','X5...","{'X5385371204','X5385374175','X5385353666','X5...","{2247,2100,2000,2000,1751,1443,1212,1205,1205,...","{2350,2378,2409,2497,2514,2559,2590,2600,2657,...","{500,5000,800,1600,1000,700,5000,5200,800,2200}","{10000,2000,5000,1000,200,500,300,10000,4000,500}",2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,26.66
1,NX_426238,PH-20240101-04,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:56.947 +0000,2023-12-31 23:00:00.000 +0000,"{'X5385372330','X5385372495','X5385371311','X5...","{'X5385372323','X5385372333','X5385373948','X5...","{2061,2061,2051,2000,2000,2000,1920,1467,1000,...","{2251,2251,2262,2299,2300,2433,2447,2448,2551,...","{1900,3000,100,5000,800,1500,400,500,1000,1000}","{200,500,200,3000,10000,500,3600,15000,4000,4000}",2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00,24.48
2,NX_426245,PH-20240101-05,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:00.755 +0000,2023-12-31 23:00:00.000 +0000,"{'X5385359141','X5385367235','X5385367212','X5...","{'X5385364822','X5385364828','X5385364821','X5...","{2000,1950,1927,1917,1916,1320,1319,1319,1319,...","{2127,2127,2129,2150,2200,2400,2400,2401,2401,...","{100,100,1200,9000,2000,4500,1600,500,1000,1000}","{200,5000,400,10000,2000,4000,5000,500,15000,400}",2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00,24.01
3,NX_426252,PH-20240101-06,2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:06.307 +0000,2023-12-31 23:00:00.000 +0000,"{'X5385371584','X5385371589','X5385371590','X5...","{'X5385363190','X5385363118','X5383614807','X5...","{1574,1574,1574,1574,1574,1574,1424,1423,1000,...","{1798,1800,1875,2050,2123,2423,2504,2550,2670,...","{1500,5000,1000,2000,1000,5000,2000,5000,1000,...","{10000,10000,10000,5000,15000,1000,1900,10000,...",2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00,21.23
4,NX_426260,PH-20240101-07,2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:10.161 +0000,2023-12-31 23:00:00.000 +0000,"{'X5384463871','X5385167819','X5385169379','X5...","{'X5385370820','X5385371596','X5385371593','X5...","{1748,1748,1748,1748,1746,1520,1500,1500,1362,...","{2039,2089,2090,2262,2462,2562,2670,2700,2710,...","{4900,1000,1000,500,5000,1000,2000,1300,5000,5...","{500,1000,10000,15000,10000,1000,5000,5000,100...",2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00,22.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9883135,NX_496910,PH-20250101-20,2025-01-01 18:00:00+00:00,2025-01-01 19:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:44.288 +0000,2024-12-31 22:59:00.000 +0000,"{'X10233448499','X10234051602','X10234052934',...","{'X10232980063','X10228851570','X10232667440',...","{980,828,828,809,743,721,721,658,575,545}","{2775,2780,2780,2810,2810,2870,2900,3500}","{3900,4000,5000,5000,5000,5000,4000,800,2000,200}","{200,8000,5000,5000,5000,10000,2700,10000}",NaT,NaT,NaN
9883136,NX_496918,PH-20250101-21,2025-01-01 19:00:00+00:00,2025-01-01 20:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:48.436 +0000,2024-12-31 22:59:00.000 +0000,"{'X10233744528','X10233744531','X10233744825',...","{'X10228851573','X10229998186','X10229998185',...","{513,513,509,500,498,498,497,394,377,376}","{2780,2780,2810,2810,2820,2870,2900,2900,2950,...","{5000,5000,5000,3900,5000,5000,300,2500,5000,6...","{6400,5000,5000,5000,10000,10000,1900,5000,700...",NaT,NaT,NaN
9883137,NX_496925,PH-20250101-22,2025-01-01 20:00:00+00:00,2025-01-01 21:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:52.589 +0000,2024-12-31 22:59:00.000 +0000,"{'X10234238392','X1023

## Remove rows where DA price is NaN
DA prices go a little bit further than the orderbook. Remove these

In [62]:
df = df.dropna(subset=['NO3 Price (EUR)'])

## Write merged orderbook to parquet

In [63]:
df.to_parquet('Data/merged_orderbook.parquet')

## Start here

In [81]:
df = pd.read_parquet('Data/merged_orderbook.parquet')

In [66]:
df.dtypes

contractId                           object
contractName                         object
dlvryStart              datetime64[ns, UTC]
dlvryEnd                datetime64[ns, UTC]
dlvryAreaId                           int64
marketId                             object
area                                 object
recorded                             object
time                    datetime64[ns, UTC]
orderId_bid                          object
orderId_ask                          object
orderPrice_bid                       object
orderPrice_ask                       object
orderQty_bid                         object
orderQty_ask                         object
NO3 Price (EUR)                     float64
Delivery Start (UTC)    datetime64[ns, UTC]
Delivery End (UTC)      datetime64[ns, UTC]
dtype: object

## Formatting the DF

In [82]:
# Set the delivery hour to be the index
def setIndex(df):
    df = df.set_index('dlvryStart')
    return df

# Sort by dlvryStart and time. Meaning that we get dlvryStart at e.g. 23 with time=14.01, 14.02 etc before dlvryStart at 00 with time=14.01, 14.02 etc
def sortByDlvryStart(df):
    # Sort by time within each index group
    df = df.sort_index()
    df = df.groupby(df.index, group_keys=False).apply(lambda x: x.sort_values(by='time'))
    return df

# Drop unnecessary columns
def dropColumns(df):
    df = df.drop(columns=['contractId', 'contractName', 'dlvryEnd', 'dlvryAreaId', 'marketId', 'area', 'recorded', 'Delivery Start (UTC)', 'Delivery End (UTC)', 'orderId_bid', 'orderId_ask'])
    return df

# Extract first element in a string. To be used in retrieveBestOffers function
def extractFirstElement(text):
    match = re.search(r'{(\d+)', str(text))
    return float(match.group(1)) if match else 0

# The prices and quantities need to be converted to list to retrieve the first value
def retrieveBestOffers(df):
    columns = ['orderPrice_bid', 'orderQty_bid', 'orderPrice_ask', 'orderQty_ask']
    for col in columns:
        df[col] = df[col].apply(lambda x: extractFirstElement(x))
    return df

# Get ID prices at the same magnitude as DA prices
def fixPriceVolumeMagnitude(df):
    df['orderPrice_bid'] = df['orderPrice_bid'] / 100
    df['orderPrice_ask'] = df['orderPrice_ask'] / 100
    df['orderQty_bid'] = df['orderQty_bid'] / 1000
    df['orderQty_ask'] = df['orderQty_ask'] / 1000
    return df

# Rename column names for better readability
def changeName(df):
    df = df.rename(columns={'NO3 Price (EUR)': 'DA_price'})
    return df

# Split delivery hour and time to month, day, hour etc
def createNewColumns(df):
    df['dlvryMonth'] = df.index.month
    df['dlvryDay'] = df.index.day
    df['dlvryHour'] = df.index.hour
    df['dlvryWeekday'] = df.index.weekday

    df['timeMonth'] = df['time'].dt.month
    df['timeDay'] = df['time'].dt.day
    df['timeHour'] = df['time'].dt.hour
    df['timeWeekday'] = df['time'].dt.weekday
    df['timeMinute'] = df['time'].dt.minute
    return df





In [ ]:
#df = setIndex(df)
#df = sortByDlvryStart(df)
#df = dropColumns(df)
#df = retrieveBestOffers(df)
#df = fixPriceVolumeMagnitude(df)
#df = changeName(df)
#df = createNewColumns(df)

In [97]:
df

,time,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask,DA_price,dlvryMonth,dlvryDay,dlvryHour,dlvryWeekday,timeMonth,timeDay,timeHour,timeWeekday,timeMinute
dlvryStart,,,,,,,,,,,,,,,
2024-01-01 01:00:00+00:00,2023-12-31 23:00:00+00:00,22.47,23.50,0.5,10.0,26.66,1,1,1,0,12,31,23,6,0
2024-01-01 01:00:00+00:00,2023-12-31 23:01:00+00:00,22.47,24.12,0.5,0.5,26.66,1,1,1,0,12,31,23,6,1
2024-01-01 01:00:00+00:00,2023-12-31 23:02:00+00:00,22.47,24.20,0.5,9.1,26.66,1,1,1,0,12,31,23,6,2
2024-01-01 01:00:00+00:00,2023-12-31 23:03:00+00:00,22.47,24.59,0.5,0.3,26.66,1,1,1,0,12,31,23,6,3
2024-01-01 01:00:00+00:00,2023-12-31 23:04:00+00:00,22.47,24.29,0.5,0.6,26.66,1,1,1,0,12,31,23,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 22:00:00+00:00,2024-12-31 20:55:00+00:00,12.62,16.85,4.3,1.5,12.85,12,31,22,1,12,31,20,1,55
2024-12-31 22:00:00+00:00,2024-12-31 20:56:00+00:00,12.62,16.85,4.3,1.5,12.85,12,31,22,1,12,31,20,1,56
2024-12-31 22:00:00+00:00,2024-12-31 20:57:00+00:00,12.62,16.85,4.3,1.5,12.85,12,31,22,1,12,31,20,1,57


## Train/Test split

In [108]:
train = df.loc[df.index < '2024-10-01 00:00:00']
test = df.loc[df.index >= '2024-10-01 00:00:00']

In [ ]:
FEATURES = ['orderPrice_bid', 'orderQty_bid', 'orderPrice_ask', 'orderQty_ask', 'DA_price', 'dlvryMonth', 'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour', 'timeWeekday', 'timeMinute']
TARGETS = 

,time,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask,DA_price,dlvryMonth,dlvryDay,dlvryHour,dlvryWeekday,timeMonth,timeDay,timeHour,timeWeekday,timeMinute
dlvryStart,,,,,,,,,,,,,,,
2024-10-01 00:00:00+00:00,2024-09-30 12:00:00+00:00,0.00,0.00,0.0,0.0,10.96,10,1,0,1,9,30,12,0,0
2024-10-01 00:00:00+00:00,2024-09-30 12:01:00+00:00,0.00,0.00,0.0,0.0,10.96,10,1,0,1,9,30,12,0,1
2024-10-01 00:00:00+00:00,2024-09-30 12:02:00+00:00,0.00,0.00,0.0,0.0,10.96,10,1,0,1,9,30,12,0,2
2024-10-01 00:00:00+00:00,2024-09-30 12:03:00+00:00,0.00,0.00,0.0,0.0,10.96,10,1,0,1,9,30,12,0,3
2024-10-01 00:00:00+00:00,2024-09-30 12:04:00+00:00,0.00,0.00,0.0,0.0,10.96,10,1,0,1,9,30,12,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 22:00:00+00:00,2024-12-31 20:55:00+00:00,12.62,16.85,4.3,1.5,12.85,12,31,22,1,12,31,20,1,55
2024-12-31 22:00:00+00:00,2024-12-31 20:56:00+00:00,12.62,16.85,4.3,1.5,12.85,12,31,22,1,12,31,20,1,56
2024-12-31 22:00:00+00:00,2024-12-31 20:57:00+00:00,12.62,16.85,4.3,1.5,12.85,12,31,22,1,12,31,20,1,57
